In [ ]:
from pathlib import Path
import json

import pandas as pd

from consent.util.default_path import get_data_dir
from ooutil.df_util import read_jsons_to_df
from ooutil.file import file_empty


# data_dir = get_data_dir('2021-03-21/pref_btn_scan')
data_dir = get_data_dir('2021-04-25/pref_btn_scan')

In [3]:
json_files = []
for json_file in data_dir.glob('**/scan.json'):
    # json_file = adir / (adir.name + '.json')
    if not json_file.exists() or file_empty(json_file):
        print(f'{json_file=} not exist')
        continue
    json_files.append(json_file)

In [4]:
df = read_jsons_to_df(json_files)
df

,domain,url,failed,failed_reason,pref_btn_selectors,consent_lib,html
0,nzherald.co.nz,https://nzherald.co.nz,False,None,[],None,"<!DOCTYPE html><html><head><script type=""text/..."
1,obsproject.com,https://obsproject.com,False,None,[],None,"<!DOCTYPE html><html lang=""en"" dir=""ltr""><head..."
2,airtel.in,https://airtel.in,False,None,[],None,"<!DOCTYPE html><html lang=""EN-US""><head><title..."
3,nrdc.org,https://nrdc.org,False,None,[],onetrust_legacy,"<!DOCTYPE html><html lang=""en"" dir=""ltr"" prefi..."
4,square.site,https://square.site,False,None,[],None,"<!DOCTYPE html><html class=""country-US lang-en..."
...,...,...,...,...,...,...,...
3316,marketwatch.com,https://marketwatch.com,False,None,[],None,"<!DOCTYPE html><html class=""enhanced icons-loa..."
3317,nodejs.org,https://nodejs.org,False,None,[],None,"<!DOCTYPE html><html lang=""en""><head>\n <meta..."
3318,wikidot.com,https://wikidot.com,False,None,[],None,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
3319,paytm.in,https://paytm.in,False,None,[],None,"<!DOCTYPE html><html lang=""en"" class=""no-js""><..."


In [5]:
df = df.dropna(subset=['failed', 'pref_btn_selectors'])
df = df.fillna('')

In [6]:
pref_btn_sites = df[df.pref_btn_selectors.map(len) > 0]
pref_btn_sites

,domain,url,failed,failed_reason,pref_btn_selectors,consent_lib,html
16,brealtime.com,https://www.brealtime.com,False,,[#menu-item-434 #icon-id0003944390512251594],,"<!DOCTYPE html><html lang=""en-US""><head><style..."
35,blackberry.com,https://blackberry.com,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html lang=""en_us""><head>\n ..."
36,siemens.com,https://siemens.com,False,,[.ot-sdk-row > #onetrust-button-group-parent #...,onetrust,"<!DOCTYPE html><html lang=""en""><head><meta nam..."
46,healthgrades.com,https://healthgrades.com,False,,[.ot-sdk-row > #onetrust-button-group-parent #...,onetrust,"<!DOCTYPE html><html lang=""en""><head>\n <..."
47,xoom.com,https://xoom.com,False,,"[.col-sm-12 li:nth-child(5) > a, .row .xvx-fon...",,"<!DOCTYPE html><html lang=""en-US"" class="" ""><h..."
...,...,...,...,...,...,...,...
3291,teads.tv,https://teads.tv,False,,"[span .cli_settings_button, span #cookie_actio...",,"<!DOCTYPE html><html lang=""en-US"" prefix=""og: ..."
3298,epicurious.com,https://epicurious.com,False,,[.corporate-info-wrap #ot-sdk-btn],onetrust,"<!DOCTYPE html><html lang=""en""><head>\n <me..."
3304,lolesports.com,https://lolesports.com,False,,[.group-images-copyright .link:nth-child(3)],osano,"<!DOCTYPE html><html lang=""en-US""><head><style..."
3309,bestwestern.com,https://bestwestern.com,False,,[.ot-sdk-row > #onetrust-button-group-parent #...,onetrust,"<!DOCTYPE html><html lang=""en-us""><head>\n\n\n..."


In [15]:
onetrust_sites = set(pref_btn_sites[pref_btn_sites.consent_lib.str.contains('onetrust')].domain.tolist())

print("Number onetrust pref", len(pref_btn_sites[pref_btn_sites.consent_lib == 'onetrust']))
print("Number onetrust + legacy pref", len(onetrust_sites))

Number onetrust pref 230
Number onetrust + legacy pref 245


In [19]:
tcf_sites_file = Path('/home/ducbui/Dropbox/projects/optout/data/2021-03-07/tcf_topsites/tcf_sites.json')
tcf_sites = pd.read_json(tcf_sites_file)
tcf_domains = set(tcf_sites.domain.tolist())

onetrust_sites_with_tcf = onetrust_sites.intersection(tcf_domains)
print(f"Number scanned with tcf: {len(onetrust_sites_with_tcf)}")

Number scanned with tcf: 9


In [8]:
out_file = data_dir / 'pref_btn_sites.csv'
# Uncomment to save
# pref_btn_sites.to_csv(out_file, index=False); print(f"Written pref_btn_sites to {out_file}")

In [9]:
pref_btn_sites.consent_lib.value_counts()

onetrust           230
                   185
trustarc            16
onetrust_legacy     15
cookiebot            4
quantcast            2
osano                2
google               1
Name: consent_lib, dtype: int64

In [10]:
pref_btn_sites[pref_btn_sites.consent_lib.str.contains('onetrust_legacy')].reset_index(drop=True)


,domain,url,failed,failed_reason,pref_btn_selectors,consent_lib,html
0,blackberry.com,https://blackberry.com,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html lang=""en_us""><head>\n ..."
1,fifa.com,https://fifa.com,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html lang=""en-GB"" dir=""ltr""><h..."
2,trulia.com,https://trulia.com,False,,[.FooterLinkList__Container-yuht23-0 .Box-sc-1...,onetrust_legacy,"<!DOCTYPE html><html lang=""en"" class=""js-focus..."
3,pantheonsite.io,https://www.pantheonsite.io,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html lang=""en"" dir=""ltr"" class..."
4,cancer.org,https://cancer.org,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html xmlns=""http://www.w3.org/..."
5,usabilla.com,https://usabilla.com,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html class=""no-js"" lang=""en""><..."
6,cylance.com,https://cylance.com,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html lang=""en_us""><head>\n ..."
7,docusign.com,https://docusign.com,False,,"[.container .optanon-toggle-display, .optanon-...",onetrust_legacy,"<!DOCTYPE html><html lang=""en"" dir=""ltr"" prefi..."
8,ap.org,https://ap.org,False,,"[.u-flex-lg .optanon-toggle-display, .optanon-...",onetrust_legacy,"<!DOCTYPE html><html lang=""en"" class=""js svg"">..."
9,entrust.net,https://entrust.net,False,,[.optanon-alert-box-button-container .optanon-...,onetrust_legacy,"<!DOCTYPE html><html lang=""en""><head>\n<title>..."


In [11]:
pref_btn_sites[pref_btn_sites.consent_lib == ''].reset_index(drop=True)


,domain,url,failed,failed_reason,pref_btn_selectors,consent_lib,html
0,brealtime.com,https://www.brealtime.com,False,,[#menu-item-434 #icon-id0003944390512251594],,"<!DOCTYPE html><html lang=""en-US""><head><style..."
1,xoom.com,https://xoom.com,False,,"[.col-sm-12 li:nth-child(5) > a, .row .xvx-fon...",,"<!DOCTYPE html><html lang=""en-US"" class="" ""><h..."
2,colorado.gov,https://colorado.gov,False,,[.skin-default #views_slideshow_controls_text_...,,"<!DOCTYPE html><html class=""js js flexbox flex..."
3,rawstory.com,https://rawstory.com,False,,[.main p:nth-child(2) > .footer-link],,"<!DOCTYPE html><html lang=""en"" class=""""><head ..."
4,doaj.org,https://doaj.org,False,,[.homepage > #cookie-consent #cookie-consent-h...,,"<!DOCTYPE html><html dir=""ltr"" lang=""en""><head..."
...,...,...,...,...,...,...,...
180,sportbible.com,https://sportbible.com,False,,[.css-ti30lp-padding-padding-sectionBorder #po...,,"<!DOCTYPE html><html lang=""en""><head><script s..."
181,emerald.com,https://emerald.com,False,,[.bg-dark > #feedback-strip .header-feedback__...,,"<!DOCTYPE html><html lang=""en"" xmlns:mml=""http..."
182,independent.ie,https://independent.ie,False,,[.didomi-popup-container #didomi-notice-learn-...,,"<!DOCTYPE html><html lang=""en"" js="""" r5s="""" cl..."
183,ladbible.com,https://ladbible.com,False,,[.css-wazu5f-padding-padding-sectionBorder #po...,,"<!DOCTYPE html><html lang=""en""><head><script s..."
